# Recommender Systems
There are basically 2 Types of Recommender Systems - 


1.   Content Based
2.   Collabrative Filtering

Both of them are quite Self Explanatory by their names, but in short we can understand each one as following - 



1.   Content Based

![alt text](https://miro.medium.com/max/625/1*BME1JjIlBEAI9BV5pOO5Mg.png)

2.   Collabrative Filtering

![alt text](https://miro.medium.com/max/625/1*x8gTiprhLs7zflmEn1UjAQ.png)

For more Theoritical Info regarding, you can check out some of the links I have shared in the Ending of the Note Book.






# Content Based Recommender System With NLP in NLTK

In [228]:
# importing dependencies

# General Analysis Dependencies (I have a habit of importing as they are great, even not in use)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import csv

# More dependencies for this specific Recommender system for NLP and pre processing of data
! pip install rake_nltk
import nltk
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
# importing data set of 250 Top rated movies, from the given link
data = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

In [230]:
# lets have a look at the data set we have
data.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [231]:
# in this Recommendation system we will be using only the features of Actors, Directors, Plot, Genre and Title
df = data[['Title', 'Genre', 'Actors', 'Director', 'Plot']]
df.head()

,Title,Genre,Actors,Director,Plot
0,The Shawshank Redemption,"Crime, Drama","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Frank Darabont,Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama","Marlon Brando, Al Pacino, James Caan, Richard ...",Francis Ford Coppola,The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama","Al Pacino, Robert Duvall, Diane Keaton, Robert...",Francis Ford Coppola,The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama","Christian Bale, Heath Ledger, Aaron Eckhart, M...",Christopher Nolan,When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",Sidney Lumet,A jury holdout attempts to prevent a miscarria...


In [232]:
# lower casing all data for easiness and no duplication of data in any case
for index, row in df.iterrows():
  
  actors = row['Actors']
  row['Actors'] = actors.lower()
  
  directors = row['Director']
  row['Director'] = directors.lower()
  
  genre = row['Genre']
  row['Genre'] = genre.lower()
  
  plot = row['Plot']
  row['Plot'] = plot.lower()
  
df.head()

,Title,Genre,Actors,Director,Plot
0,The Shawshank Redemption,"crime, drama","tim robbins, morgan freeman, bob gunton, willi...",frank darabont,two imprisoned men bond over a number of years...
1,The Godfather,"crime, drama","marlon brando, al pacino, james caan, richard ...",francis ford coppola,the aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"crime, drama","al pacino, robert duvall, diane keaton, robert...",francis ford coppola,the early life and career of vito corleone in ...
3,The Dark Knight,"action, crime, drama","christian bale, heath ledger, aaron eckhart, m...",christopher nolan,when the menace known as the joker emerges fro...
4,12 Angry Men,"crime, drama","martin balsam, john fiedler, lee j. cobb, e.g....",sidney lumet,a jury holdout attempts to prevent a miscarria...


In [233]:
# Appending all , seperated data to a common column
df['Common'] = ""

for index, row in df.iterrows():
  actors = row['Actors']
  directors = row['Director']
  genre = row['Genre']
  
  row['Common'] = actors + ', ' + directors + ', ' + genre
  
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Title,Genre,Actors,Director,Plot,Common
0,The Shawshank Redemption,"crime, drama","tim robbins, morgan freeman, bob gunton, willi...",frank darabont,two imprisoned men bond over a number of years...,"tim robbins, morgan freeman, bob gunton, willi..."
1,The Godfather,"crime, drama","marlon brando, al pacino, james caan, richard ...",francis ford coppola,the aging patriarch of an organized crime dyna...,"marlon brando, al pacino, james caan, richard ..."
2,The Godfather: Part II,"crime, drama","al pacino, robert duvall, diane keaton, robert...",francis ford coppola,the early life and career of vito corleone in ...,"al pacino, robert duvall, diane keaton, robert..."
3,The Dark Knight,"action, crime, drama","christian bale, heath ledger, aaron eckhart, m...",christopher nolan,when the menace known as the joker emerges fro...,"christian bale, heath ledger, aaron eckhart, m..."
4,12 Angry Men,"crime, drama","martin balsam, john fiedler, lee j. cobb, e.g....",sidney lumet,a jury holdout attempts to prevent a miscarria...,"martin balsam, john fiedler, lee j. cobb, e.g...."


In [234]:
# dropping the Genre column
df.drop(columns = ['Genre'], inplace = True)

# dropping the Actors column
df.drop(columns = ['Actors'], inplace = True)

# dropping the Director column
df.drop(columns = ['Director'], inplace = True)

df.tail()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Title,Plot,Common
245,The Lost Weekend,the desperate life of a chronic alcoholic is f...,"ray milland, jane wyman, phillip terry, howard..."
246,Short Term 12,a 20-something supervising staff member of a r...,"brie larson, john gallagher jr., stephanie bea..."
247,His Girl Friday,a newspaper editor uses every trick in the boo...,"cary grant, rosalind russell, ralph bellamy, g..."
248,The Straight Story,an old man makes a long journey by lawn-mover ...,"sissy spacek, jane galloway heitz, joseph a. c..."
249,Slumdog Millionaire,a mumbai teen reflects on his upbringing in th...,"dev patel, saurabh shukla, anil kapoor, raj zu..."


In [235]:
# also we have to make the names like anil kapoor --> anilkapoor to remove chances of First name or last name common problem
# Second step we convert ',' to ' '
for i in range(0, len(df) - 1) : 
  df.iloc[i]['Common'] = df.iloc[i]['Common'].replace(' ', '')
  df.iloc[i]['Common'] = df.iloc[i]['Common'].replace(',', ' ')
df.head()
 

,Title,Plot,Common
0,The Shawshank Redemption,two imprisoned men bond over a number of years...,timrobbins morganfreeman bobgunton williamsadl...
1,The Godfather,the aging patriarch of an organized crime dyna...,marlonbrando alpacino jamescaan richards.caste...
2,The Godfather: Part II,the early life and career of vito corleone in ...,alpacino robertduvall dianekeaton robertdeniro...
3,The Dark Knight,when the menace known as the joker emerges fro...,christianbale heathledger aaroneckhart michael...
4,12 Angry Men,a jury holdout attempts to prevent a miscarria...,martinbalsam johnfiedler leej.cobb e.g.marshal...


In [0]:
# now we have to do the same with plot, but remove stop words using NLTK Rake
for index, row in df.iterrows():
  plot = row['Plot']

  # using Rake to remove the stop words from NLTK lib
  # and discards all puntuation characters as well
  r = Rake()

  # extracting the words by passing the text
  r.extract_keywords_from_text(plot)

  # getting the dictionary with key words as keys and their scores as values
  key_words_dict_scores = r.get_word_degrees()
  
  # for i in range(0, len() - 1)
  
  str1 = ' '
  
  str1 = str1.join(list(key_words_dict_scores.keys()))
  
  row['Common'] = row['Common'] + str1


In [237]:

# dropping the Plot column
df.drop(columns = ['Plot'], inplace = True)

df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Title,Common
0,The Shawshank Redemption,timrobbins morganfreeman bobgunton williamsadl...
1,The Godfather,marlonbrando alpacino jamescaan richards.caste...
2,The Godfather: Part II,alpacino robertduvall dianekeaton robertdeniro...
3,The Dark Knight,christianbale heathledger aaroneckhart michael...
4,12 Angry Men,martinbalsam johnfiedler leej.cobb e.g.marshal...


Now is the time for the model but before designing our model, lets take a look at what we have done so far

*   We Imported Data
*   Then cleaned the data in a Features column for each corresponding movie

So we now have some vectors to play around with.

We will be using **Cosine Similiarity** for these vectors, lets take a look at it - 

**Dot Product -**

![alt text](https://miro.medium.com/max/875/1*xD_jLdpqESuOaBFXyV9YGA.png)

**Cosine Similiarity -**

![alt text](https://miro.medium.com/max/875/1*r5ULMbx7ju3_Y4TU1PJIyQ.png)

Both the Diagrams are self Explanatory, so lets move on to designing our model and converting our vectors to Integer values for Mathematical Operations - 

In [238]:
# Now to Continue we need to Vectorize the Common column, We have 2 options - 
# CountVectorizer or TfIdfVectorizer.
# TfIdfVectorizer(give less imp to more freq ones) takes into consideration the value of each word also, in our case we have custom type of words so we will use the other one


# generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['Common'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.10259784, 0.09176629, ..., 0.        , 0.        ,
        0.        ],
       [0.10259784, 1.        , 0.26832816, ..., 0.        , 0.        ,
        0.        ],
       [0.09176629, 0.26832816, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

# Similiarity Matrix

The similiarity matrix will look like as show below - 

![alt text](https://miro.medium.com/max/533/1*CQ-1nBzzcGJ9ncEMgj1_lA.png)

**Implications-**

1.   The diagonal values are 1 as each film is similiar to itself
2.   Secondle the matrix is symmetric i.e. A = B -> B = A

In [239]:
# now we can have our recommendation engine to work, it will give me top n movies with maximum value in similiar to the input movie

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)

print(len(df))

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies (you can change 10 to any value)
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # getting index of title of movie passed
    for i in range(0, len(df) - 1):
      if df.iloc[i]['Title'] == title:
        idx = i
      
    
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        print(df.iloc[i]['Title'])
        

250


In [240]:
# so lets see our recommendations
recommendations('The Dark Knight')

Batman Begins
The Dark Knight Rises
The Prestige
Guardians of the Galaxy Vol. 2
Baby Driver
The Avengers
Sin City
Spider-Man: Homecoming
Witness for the Prosecution
Rush


# Collabrative Filtering approach

With the content based, it seemed that one may get in a loop to see similiar movies with same cast members and genre. So to overcome this another approach is the Collabrative Filtering approach as described earlier. 



In [4]:
# Upload movie.csv from MovieLens Dataset
from google.colab import files
uploads = files.upload()

Saving movies.csv to movies.csv


In [5]:
# Upload ratings.csv from MovieLens Dataset
from google.colab import files
uploads = files.upload()

Saving ratings.csv to ratings.csv


In [0]:
# importing dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import sklearn

**Collabrative Filtering can be performed in 2 ways -**


1.   Item Based (Preferred)
2.   User Based

Item based is better than User based due to the Dynamic nature of users, while items remains same after they are defined.

For Item based, we will use KNN model.



In [7]:
# Building data frames
df_movie = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')

,userId,movieId,rating,timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [8]:
df_movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
df_ratings.tail()

,userId,movieId,rating,timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


# KNN Working

**KNN works based on Item based Similiarity Model, which can be visualised in following manner -**

![alt text](https://miro.medium.com/max/813/1*OyYyr9qY-w8RkaRh2TKo0w.png)

KNN will calculate the “distance” between the target movie and every other movie in its database, then it ranks its distances and returns the top K nearest neighbor movies as the most similar movie recommendations.

To make the data feedable to KNN model, we will convert the data to say a mxn matrix with say m > n. Then we pivot the matrix to mxm with 0s in place of missing spaces (added after pivoting). 

*Note - m -> movies, n -> Users*

In [0]:
# import new dependencie Scipy for pivoting 
from scipy.sparse import csr_matrix

In [0]:
# new data frame (pivoted)
df_movie_features = df_ratings.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)

# convert dataframe of movie features to scipy sparse matrix
mat_movie_features = csr_matrix(df_movie_features.values)

In [13]:
# After pivoting we have the following data
df_movie_features.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.5,3.5,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.5,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,2.5,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


**NOTE**

So now for calcularting the nearest distance and calculating scores according to the distance, we can use the Eucledian Formula to calculate. But our matrix dimensions are too high, so the KNN model will give poor results due to high dimensionality. To overcome this we will use Cosine Similiarity (Similiar as we performed earlier in Content based)

In [0]:
# importing our model in sklearn
from sklearn.neighbors import NearestNeighbors

# designing model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1) # Cosine similiarity


In [0]:
# function to make recommendations
def make_recommendations(self, fav_movie, n_recommendations):
    
    # get data
    movie_user_mat_sparse, hashmap = self._prep_data()
    
    # get recommendations
    raw_recommends = self._inference(
        self.model, movie_user_mat_sparse, hashmap,
        fav_movie, n_recommendations)
    # print results
    reverse_hashmap = {v: k for k, v in hashmap.items()}
    print('Recommendations for {}:'.format(fav_movie))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance '
              'of {2}'.format(i+1, reverse_hashmap[idx], dist))

In [0]:
# there are some errors, will check on this later
make_recommendations("Iron Man", 10)

# Conclusion

So we have gone through, 2 of the most basic Recommender systems. There is a third one called Hybrid which uses the combination of both. There are various out there with different feature matrix to perform the same task. It just depends on the Problem we are solving.

**Links to Sources I used and Credits to them -**


1.   https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea
2.   https://github.com/emmagrimaldi/Content_based_movie_recommender/blob/master/EG_project5.ipynb
3.   https://hackernoon.com/introduction-to-recommender-system-part-1-collaborative-filtering-singular-value-decomposition-44c9659c5e75
4.   https://www.youtube.com/watch?v=39vJRxIPSxw

